In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.2 MB/s eta 0:00:00


#Чтение, подготовка, обработка данных

In [ ]:
import pandas as pd

df = pd.read_csv('augmented_request_type.csv')

In [ ]:
df['Описание'] = df['Описание'].str.replace('_x000D_', '')

In [ ]:
df = df.drop(df[df['Точка отказа'] == 'Консультация'].index)

In [ ]:
df['Точка отказа'] = df['Точка отказа'].replace({'Диск ': 'Диск'})

Излечение признаков из текста

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

def embed_text(texts, model):
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=32, device='cuda')
    return embeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

tqdm.pandas()

tqdm.pandas(desc="Эмбеддинг текстов")
X_theme = embed_text(df['Тема'].values, embedding_model).tolist()
X_desc = embed_text(df['Описание'].values, embedding_model).tolist()

df['Тема_emb'] = X_theme
df['Описание_emb'] = X_desc

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
import gc
import torch

del embedding_model
torch.cuda.empty_cache()
gc.collect()

7167

Создание выборок для обучения и теста

In [ ]:
import numpy as np

X = np.hstack([X_theme, X_desc])

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Точка отказа'])

class_labels = label_encoder.classes_

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Обучение модели классификации точки отказа

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=5000,
    learning_rate=0.005,
    depth=6,
    random_seed=42,
    auto_class_weights='Balanced',
    verbose=200,
    task_type="GPU",
    devices='0'
)

In [ ]:
model.fit(X_train, y_train)

0:	learn: 2.8229706	total: 229ms	remaining: 19m 5s
200:	learn: 1.5912245	total: 25s	remaining: 9m 56s
400:	learn: 1.0327504	total: 51.9s	remaining: 9m 55s
600:	learn: 0.7391327	total: 1m 20s	remaining: 9m 46s
800:	learn: 0.5727538	total: 1m 46s	remaining: 9m 20s
1000:	learn: 0.4669973	total: 2m 12s	remaining: 8m 48s
1200:	learn: 0.3976016	total: 2m 37s	remaining: 8m 17s
1400:	learn: 0.3467241	total: 3m 1s	remaining: 7m 46s
1600:	learn: 0.3087380	total: 3m 25s	remaining: 7m 16s
1800:	learn: 0.2799566	total: 3m 48s	remaining: 6m 46s
2000:	learn: 0.2557036	total: 4m 11s	remaining: 6m 16s
2200:	learn: 0.2349951	total: 4m 34s	remaining: 5m 49s
2400:	learn: 0.2191576	total: 4m 57s	remaining: 5m 21s
2600:	learn: 0.2040566	total: 5m 20s	remaining: 4m 55s
2800:	learn: 0.1911731	total: 5m 42s	remaining: 4m 29s
3000:	learn: 0.1803593	total: 6m 6s	remaining: 4m 3s
3200:	learn: 0.1705656	total: 6m 29s	remaining: 3m 38s
3400:	learn: 0.1611488	total: 6m 52s	remaining: 3m 13s
3600:	learn: 0.1531786	to

Замеры метрик

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")

Accuracy: 0.8410
Precision: 0.8494
Recall: 0.8410
F1 Score (weighted): 0.8404


Сохранение модели

In [ ]:
model.save_model('request_type_classifier.cbm', format='cbm')